# Hamiltonian descent for Neural Networks

Many commonly used optimization methods in Machine Learnign (*ML*) show linear to sublinear convergence rates. Hamiltonian Descent Methods (*HD*) are a new method that extend linear convergence for a broader class of convex functions (Maddison et al., 2018). We provide a method that implements HD in TensorFlow. We compare the method's performance to stochastic gradient descent (*SGD*) and adaptive moment estimation (*Adam*). We analyze performance using a convolutional neural network (*CNN*) using the MNIST dataset as well as using <span style="color:red">...</span> .

---

This submission has mainly been prepared by Jannis Zeller (RWTH Aachen University), with help from David Schischke (University of Trier). 

## Table of Contents 

* [Theoretical Background](#theoretical-background)
* [Implementation in TensorFlow](#implementation-in-tensorflow)
* [Performance Analysis](#performance-analysis)
  * [CNN: MNIST](#cnn-mnist)
* [Discussion](#disccusion)
* [Literature](#literature)

## Theoretical Background 

This section provides a brief summary of the HD methods first proposed by [Maddison et al. (2018)](https://arxiv.org/abs/1809.05042), adding some context and comparisons to commonly used optimization methods for Neural Networks. We first present a brief introduction of the terms we will use subsequently whereas, for the sake of understandability, we try to use as little mathematical notation as possible. 

### Hamiltonian Mechanics

<span style="color:orange"> @Jannis: Kurze Intro was die machen lol </span>

### Optimization basics

Optimization methods are characterized by several requirements they place on the function that needs to be optimized (*objective function*), some of which are of relevance for the understanding of HD. For more concise definitions, we refer to [Bierlaire (2015)](http://optimizationprinciplesalgorithms.com/) for a well-written and (comparably) easy to understand introductory textbook on optimization that is freely available.

* **Convexity of the Objective Function**: Convex functions are a family of functions that have desirable properties for optimization methods, mainly the fact that any local optimum we find is certain to be a global optimum as well. In brief, convex functions describe the family of functions for which the connecting line between two points is above the function values at every point. All optimization methods in this notebook are designed to work on convex functions.
* **Constrainedness of the Objective Function**: An optimization is constrained if the objective function is restricted by equality requirements (e.g. $\min f(x) \text{ s.t. } c(x) = 0$) or by inequality requirements (e.g. $\min f(x) \text{ s.t. } c(x) \leq 0$). All optimization methods presented in this notebook perform unconstrained optimization.
* **Differentiability of the Objective Function**: Optimization methods require different degrees of differentiability of the objective function. All optimization methods presented in this notebook require information about the gradient of the objective function, which means that the objective function must be differentiable once (i.e. $f \in C^1$).

In addition to the differences in requirements on the objective function, optimization algorithms differ in their efficiency in terms of iterations $k$ needed to find an optimal solution $\hat x$ (so-called *convergence rate*). We can differentiate four orders of convergence, ranked from fastest to slowest: 

1. **Quadratic**: $\lVert x^{k+1} - \hat x\rVert \leq C\lVert x^k - \hat x\rVert^2$ for some $C< \infty$
2. **Superlinear**: $\lVert x^{k+1} - \hat x \rVert \leq \alpha_k \lVert x^k - \hat x\rVert$ for $\alpha_k \downarrow 0$
3. **Linear**: $\lVert x^{k+1} - \hat x\rVert\leq \alpha \lVert x^k - \hat x \rVert$ for some $\alpha < 1$
4. **Sublinear**: $\lVert x^k - \hat x\rVert \leq \frac C{k^\beta}$, often $\beta \in \{2,1,\frac12\}$

### Hamiltonian Descent methods

This section is a summary of the HD methods proposed by Maddison et al. (2018). HD methods describe a set of first-order unconstrained optimization methods (i.e. $f \in C^1$) for convex functions. By incorporating the Hamiltonian Framework, it is possible to use the kinetic energy $k(p_t)$ (with $p_t$ being the momentum of $x$ at time $t$) and it's respective $\nabla k$ to obtain additional information about the objective function $f$. In order to be able to obtain linear convergence, the kinetic energy must be chosen proportional to the convex conjugate of $f(x): k(p) \propto f^*(p) + f^*(-p)$ (with $f^*$ being the convex conjugate of $f$, for intuition see [Le Priol, 2020](https://remilepriol.github.io/dualityviz/)). This assumption can be relaxed to $k(p)\geq \alpha \max\{f^*(p), f^*(-p)\}, \; 0 < \alpha \leq 1$ while maintaining linear convergence. Furthermore, depending on the nature of $f$, $k$ must be chosen appropriately to ensure linear convergence.

An apparent benefit of the *HD* method is that it achieves linear convergence while using a fixed step size. 




For neural networks, the most commonly used method is SGD.

* SGD: Sublinear convergence $\mathcal{O} \left(\dfrac{1}{k}\right)$ with $k = 1, ..., \infty$ being the iteration

## Implementation in TensorFlow

In [1]:
# %% Imports
#-------------------------------------------------------------------------------
import warnings
from typing import Callable

import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
#-------------------------------------------------------------------------------

# %% Implementation
# Implementation of custom tf/tf.keras optimizer inspired by https://cloudxlab.com/blog/writing-custom-optimizer-in-tensorflow-and-keras/
#-------------------------------------------------------------------------------

class KineticEnergyGradients():
    """This class provides several functions which serve as kinetic energy gradients in Hagrad.
    """

    @tf.function
    def classical(p_var: tf.Tensor) -> tf.Tensor:
        """Classical kinetic energy ||p||^2/2 with gradient p."""
        return(p_var)

    @tf.function
    def relativistic(p_var: tf.Tensor) -> tf.Tensor:
        """Relativistic kinetic energy sqrt( ||p||^2 + 1 )-1 with gradient p/sqrt( ||p||^2 + 1 )"""
        return(p_var / tf.math.sqrt(tf.math.square(tf.norm(p_var)) + 1.))

    def power(self, power_a=2., power_A=1.) -> Callable[[tf.Tensor, tf.Tensor], tf.Tensor]:
        """Power kinetic energy (1/A) * ( ||p||^a + 1 )^(A/a) - (1/A) with gradient p * ||p||^(a-2) * ( ||p||^a + 1 )^(A/a-1)"""
        a = float(power_a)
        A = float(power_A)
        @tf.function
        def power_func(p_var: tf.Tensor) -> tf.Tensor:
            #"""Power kinetic energy (1/A) * ( ||p||^a + 1 )^(A/a) - (1/A) with gradient p * ||p||^(a-2) * ( ||p||^a + 1 )^(A/a-1)"""
            p_norm = tf.norm(p_var)
            return(p_var*p_norm**(a-2.) * (p_norm**a + 1.)**(A/a-1.))
        power_func.__doc__ = f"Power kinetic energy (1/{A}) * ( ||p||^{a} + 1 )^({A}/{a}) - (1/{A}) with gradient p * ||p||^({a}-2) * ( ||p||^{a} + 1 )^({A}/{a}-1)"
        return power_func

    @tf.function
    def power_1norm(self, p_var: tf.Tensor) -> tf.Tensor:
        """Power kinetic energy (1/A) * ( ||p||^a + 1 )^(A/a) - (1/A) with gradient p * ||p||^(a-2) * ( ||p||^a + 1 )^(A/a-1) where ||p|| is the 1-norm."""
        a = self.power_a
        A = self.power_A
        p_norm = tf.norm(p_var, ord=1)
        return(tf.math.sign(p_var)*p_norm**(a-1.) * (p_norm**a + 1.)**(A/a-1.))


class Hagrad(keras.optimizers.Optimizer):
    _significant_decimals_hypers: int=4

    def __init__(self, 
        epsilon: float=1., 
        gamma:   float=10., 
        name:    str="hagrad", 
        kinetic_energy_gradient: Callable[[tf.Tensor], tf.Tensor]=KineticEnergyGradients.relativistic,
        p0_mean: float=1.,
        p0_std:  float=2.,

        **kwargs
    ): 
        """Call super().__init__() and use _set_hyper() to store hyperparameters"""
        super().__init__(name, **kwargs)
        self.kinetic_energy_gradient = kwargs.get("kinetic_energy_gradient", kinetic_energy_gradient)
        self._set_hyper("epsilon", kwargs.get("lr", epsilon)) 
        self._set_hyper("gamma", kwargs.get("gamma", gamma))
        self._set_hyper("p0_mean", kwargs.get("p0_mean", p0_mean)) 
        self._set_hyper("p0_std", kwargs.get("p0_std", p0_std))

        delta =  1. / (1. + kwargs.get("lr", epsilon)*kwargs.get("gamma", gamma) )
        n = self._significant_decimals_hypers -int(np.floor(np.log10(abs(delta))))-1
        self._set_hyper("delta", round(delta, n))

        eps_delta = epsilon*delta
        n = self._significant_decimals_hypers -int(np.floor(np.log10(abs(eps_delta))))-1
        self._set_hyper("eps_delta", round(eps_delta, n))

        if self.kinetic_energy_gradient.__name__ == "power_1norm":
            warnings.warn("The power_1norm kinetic energy gradient typically leads to divergence!")
    

    def _create_slots(self, var_list):
        """For each model variable, create the optimizer variable associated with it.
        TensorFlow calls these optimizer variables "slots".
        For momentum optimization, we need one momentum slot per model variable.
        """
        var_dtype = var_list[0].dtype.base_dtype

        ## Initializing kinetic energy for t=0.
        p0_mean = self._get_hyper("p0_mean", var_dtype)
        p0_std  = self._get_hyper("p0_std", var_dtype)
        for var in var_list:
            self.add_slot(var, "hamilton_momentum", tf.random_normal_initializer(mean=p0_mean, stddev=p0_std)) 
        ## Checkup
        # print(self._weights)


    @tf.function
    def _resource_apply_dense(self, grad, var):
        """Update the slots and perform one optimization step for one model variable
        """
        var_dtype = var.dtype.base_dtype
        p_var = self.get_slot(var, "hamilton_momentum")
        epsilon = self._get_hyper("epsilon", var_dtype)
        delta   = self._get_hyper("delta", var_dtype)
        eps_delta = self._get_hyper("eps_delta", var_dtype)


        p_var.assign(delta * p_var - eps_delta * grad)
        var.assign_add(epsilon * self.kinetic_energy_gradient(p_var))


        # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
        ## -> It seems like some (small amount of) computation time can be saved, if one does 
        #  not use conditions or a property to store different kinetic energy gradients.
        # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 


        # var.assign_add(epsilon * p_var) 

        ## DEPRECATED
        # if self.kinetic_energy == "classical":
        #     var.assign_add(epsilon * p_var)

        # if self.kinetic_energy == "relativistic":
        #     var.assign_add(epsilon * p_var / tf.math.sqrt(tf.math.square(tf.norm(p_var)) + 1.))

    def _resource_apply_sparse(self, grad, var):
        raise NotImplementedError

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            'epsilon': self._serialize_hyperparameter("epsilon"),
            'gamma':   self._serialize_hyperparameter("gamma"),
            'delta':   self._serialize_hyperparameter("delta"),
            'kinetic_energy_gradient': self.kinetic_energy_gradient.__doc__
        }

## Performance Analysis

We first analyze the performance of our Hamiltonian Descent method using a standard CNN and the MNIST dataset.

For comparison, we will use the SGD, as it is both the easiest and most implemented optimization method <span style="color:red"> (Quelle) </span> and Adam, as it is ... .

* Erwähnen, dass SGD stochatstisch ist und so zwar schneller, aber nur sublinear convergence

### CNN: MNIST

In [3]:
# MNIST als Kaggle Dataset -> Im Online Mode unter "Input -> Competitions -> MNIST" Daten hinzufügen
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input/digit-recognizer'):
    for filename in filenames:
        mnist_train = pd.read_csv(f"{dirname}/train.csv")
        mnist_test = pd.read_csv(f"{dirname}/train.csv")

### Other dataset

Vielleicht sentiment analysis mit Twitter Daten? Aber ich kenn die klassischen Benchmark Datensätze nicht :(

## Discussion

* Wäre es möglich, stochastic Hamiltonian Descent zu machen um Laufzeit zu verbessern?

## Literature

Bierlaire, M. (2015). *Optimization: Principles and Algorithms*. EPFL Press.

Le Priol, Remi. (2020). Visualizing Convex Conjugates. https://remilepriol.github.io/dualityviz/ 

Maddison, C. J., Paulin, D., Teh, Y. W., O'Donoghue, B., & Doucet, A. (2018). Hamiltonian descent methods. arXiv preprint c.